In [1]:
from torch import nn, optim, as_tensor
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.nn.init import *
from torchvision import transforms, utils, datasets, models
import cv2
from PIL import Image, UnidentifiedImageError
from pdb import set_trace
import time
import copy
from pathlib import Path
import os
import sys
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage import io, transform
from tqdm import trange, tqdm
import csv 
import glob
import dlib
import pandas as pd
import numpy as np
import random
import requests
from requests.adapters import HTTPAdapter
import torch
import hashlib
import shutil
import tempfile
from urllib.request import urlopen, Request
from facenet_pytorch import MTCNN, InceptionResnetV1, extract_face

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'Running on device: {device}')

def timming(since):
    time_elapsed=time.time()-since
    minutes=time_elapsed // 60
    seconds=time_elapsed%60
    return "%d m:%d s"%(minutes,seconds)

################3
#Global variables
###################
#Change the following path to the folder containing the images
data_dir=r"C:\Users\mihnea.andrei\Python scripts\resnet_andrei\images"
#nr_repeats is the number of random transformations that one image will suffer before identifying the face in it. 
nr_repeats=11

Running on device: cpu


In [2]:
horiz_flip=transforms.RandomHorizontalFlip(p=0.5)
color_jitter=transforms.ColorJitter(brightness=0.5,contrast=0.3)
gaussian_blur=transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5))
adj_sharpness=transforms.RandomAdjustSharpness(sharpness_factor=2)

In [3]:
since=time.time()
picture_formats=["jpg","jpeg","png","gif", "pdf", "png", "tiff"]
print_iter=10**(2)
detector=MTCNN()
for path, currentDirectory, files in os.walk(data_dir):
    print("Started extracting faces from folder %s"%(path))
    print("-"*15)
    counter=0
    nr_files=len(files)
    for file in files:
        flag=1
        ###########################
        #Change the following path to the folder that will contain only the faces.
        #########################
        save_to_path=r"C:\Users\mihnea.andrei\Python scripts\resnet_andrei\images_faces"
        file_path=os.path.join(path,file)
        #Need a folder for each person with pictures for that person (in this case 1 like we have for all romanians)
        #This is because the ImageLOader for resnet reads in the data this way.
        for i in range(1,nr_repeats+1):
            try:
                image=Image.open(file_path)
            except (ValueError,UnidentifiedImageError):
                flag=0
            save_to_suffix="//".join(file_path.split("\\")[-2:]).split(".jpg")[0]+"//"+str(i)+".jpg"
            u=random.random()
            if 0<=u<0.25:
                try:
                    image=horiz_flip(image)
                except ValueError:
                    pass
            if 0.25<=u<0.5:
                try:
                    image=color_jitter(image)
                except ValueError:
                    pass
            if 0.5<=u<0.75:
                try:
                    image=gaussian_blur(image)
                except ValueError:
                    pass
            if 0.75<=u<1:
                try:
                    image=adj_sharpness(image)
                except ValueError:
                    pass
            try:
                results=detector.detect(image)
            except (NotImplementedError,RuntimeError):
                flag=0
            if flag==1:
                try:
                    box=results[0][0]
                except TypeError:
                    flag=0
                if flag==1:        
                    for pic_type in picture_formats:
                        if pic_type in file_path:
                            save_to_path+="\\"+save_to_suffix.split("."+pic_type)[0]+".jpg"
                            break
                        else:
                            print("Picture format not accepted for %s"%(file_path))
                    extract_face(image,box,save_path=save_to_path)
                    save_to_path=r"C:\Users\mihnea.andrei\Python scripts\resnet_andrei\images_faces"
                    counter+=1
                    if counter%print_iter==0:
                        print("Completion: %.2f%% in %s"%(100*counter/(nr_files*nr_repeats),timming(since)))

Started extracting faces from folder C:\Users\mihnea.andrei\Python scripts\resnet_andrei\images
---------------
Started extracting faces from folder C:\Users\mihnea.andrei\Python scripts\resnet_andrei\images\eval
---------------


C:\Users\mihnea.andrei\Anaconda3\lib\site-packages\torch\nn\functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


Completion: 0.07% in 0 m:5 s
Completion: 0.14% in 0 m:9 s
Completion: 0.21% in 0 m:14 s
Completion: 0.28% in 0 m:17 s
Completion: 0.35% in 0 m:22 s
Completion: 0.42% in 0 m:26 s
Completion: 0.49% in 0 m:31 s
Completion: 0.56% in 0 m:35 s
Completion: 0.63% in 0 m:41 s
Completion: 0.70% in 0 m:44 s
Completion: 0.77% in 0 m:48 s
Completion: 0.84% in 0 m:53 s
Completion: 0.91% in 0 m:58 s
Completion: 0.98% in 1 m:2 s
Completion: 1.05% in 1 m:6 s
Completion: 1.12% in 1 m:11 s
Completion: 1.19% in 1 m:16 s
Completion: 1.26% in 1 m:20 s
Completion: 1.33% in 1 m:25 s
Completion: 1.40% in 1 m:30 s
Completion: 1.47% in 1 m:35 s
Completion: 1.54% in 1 m:42 s
Completion: 1.61% in 1 m:47 s
Completion: 1.68% in 1 m:51 s
Completion: 1.75% in 1 m:57 s
Completion: 1.82% in 2 m:2 s
Completion: 1.89% in 2 m:7 s
Completion: 1.96% in 2 m:11 s
Completion: 2.03% in 2 m:17 s
Completion: 2.10% in 2 m:22 s
Completion: 2.17% in 2 m:26 s
Completion: 2.24% in 2 m:30 s
Completion: 2.31% in 2 m:34 s
Completion: 2.38

Completion: 18.67% in 21 m:45 s
Completion: 18.74% in 21 m:50 s
Completion: 18.81% in 21 m:54 s
Completion: 18.88% in 21 m:59 s
Completion: 18.95% in 22 m:3 s
Completion: 19.02% in 22 m:7 s
Completion: 19.09% in 22 m:12 s
Completion: 19.16% in 22 m:16 s
Completion: 19.23% in 22 m:21 s
Completion: 19.30% in 22 m:26 s
Completion: 19.37% in 22 m:30 s
Completion: 19.44% in 22 m:35 s
Completion: 19.51% in 22 m:40 s
Completion: 19.58% in 22 m:45 s
Completion: 19.65% in 22 m:49 s
Completion: 19.72% in 22 m:55 s
Completion: 19.79% in 23 m:1 s
Completion: 19.86% in 23 m:8 s
Completion: 19.93% in 23 m:12 s
Completion: 20.00% in 23 m:18 s
Completion: 20.07% in 23 m:22 s
Completion: 20.14% in 23 m:26 s
Completion: 20.21% in 23 m:30 s
Completion: 20.28% in 23 m:35 s
Completion: 20.35% in 23 m:39 s
Completion: 20.42% in 23 m:45 s
Completion: 20.49% in 23 m:52 s
Completion: 20.56% in 23 m:57 s
Completion: 20.63% in 24 m:1 s
Completion: 20.70% in 24 m:6 s
Completion: 20.77% in 24 m:12 s
Completion: 20

Completion: 36.71% in 42 m:13 s
Completion: 36.78% in 42 m:19 s
Completion: 36.85% in 42 m:24 s
Completion: 36.92% in 42 m:30 s
Completion: 36.99% in 42 m:35 s
Completion: 37.06% in 42 m:40 s
Completion: 37.13% in 42 m:45 s
Completion: 37.20% in 42 m:49 s
Completion: 37.27% in 42 m:54 s
Completion: 37.34% in 42 m:58 s
Completion: 37.41% in 43 m:3 s
Completion: 37.48% in 43 m:8 s
Completion: 37.55% in 43 m:15 s
Completion: 37.62% in 43 m:19 s
Completion: 37.69% in 43 m:24 s
Completion: 37.76% in 43 m:29 s
Completion: 37.83% in 43 m:35 s
Completion: 37.90% in 43 m:39 s
Completion: 37.97% in 43 m:44 s
Completion: 38.04% in 43 m:49 s
Completion: 38.11% in 43 m:55 s
Completion: 38.18% in 43 m:59 s
Completion: 38.25% in 44 m:5 s
Completion: 38.32% in 44 m:10 s
Completion: 38.39% in 44 m:16 s
Completion: 38.46% in 44 m:20 s
Completion: 38.53% in 44 m:26 s
Completion: 38.60% in 44 m:30 s
Completion: 38.67% in 44 m:36 s
Completion: 38.74% in 44 m:43 s
Completion: 38.81% in 44 m:50 s
Completion:

...

Completion: 56.14% in 682 m:12 s
Completion: 56.15% in 682 m:18 s
Completion: 56.15% in 682 m:24 s
Completion: 56.16% in 682 m:29 s
Completion: 56.17% in 682 m:35 s
Completion: 56.18% in 682 m:40 s
Completion: 56.18% in 682 m:46 s
Completion: 56.19% in 682 m:52 s
Completion: 56.20% in 682 m:58 s
Completion: 56.21% in 683 m:4 s
Completion: 56.21% in 683 m:9 s
Completion: 56.22% in 683 m:15 s
Completion: 56.23% in 683 m:21 s
Completion: 56.23% in 683 m:26 s
Completion: 56.24% in 683 m:31 s
Completion: 56.25% in 683 m:36 s
Completion: 56.26% in 683 m:42 s
Completion: 56.26% in 683 m:48 s
Completion: 56.27% in 683 m:54 s
Completion: 56.28% in 683 m:59 s
Completion: 56.29% in 684 m:4 s
Completion: 56.29% in 684 m:10 s
Completion: 56.30% in 684 m:15 s
Completion: 56.31% in 684 m:20 s
Completion: 56.32% in 684 m:25 s
Completion: 56.32% in 684 m:30 s
Completion: 56.33% in 684 m:37 s
Completion: 56.34% in 684 m:42 s
Completion: 56.35% in 684 m:46 s
Completion: 56.35% in 684 m:52 s
Completion: 5

Completion: 57.98% in 703 m:32 s
Completion: 57.99% in 703 m:37 s
Completion: 58.00% in 703 m:44 s
Completion: 58.00% in 703 m:50 s
Completion: 58.01% in 703 m:55 s
Completion: 58.02% in 704 m:0 s
Completion: 58.02% in 704 m:4 s
Completion: 58.03% in 704 m:9 s
Completion: 58.04% in 704 m:12 s
Completion: 58.05% in 704 m:18 s
Completion: 58.05% in 704 m:23 s
Completion: 58.06% in 704 m:27 s
Completion: 58.07% in 704 m:31 s
Completion: 58.08% in 704 m:36 s
Completion: 58.08% in 704 m:42 s
Completion: 58.09% in 704 m:47 s
Completion: 58.10% in 704 m:52 s
Completion: 58.11% in 704 m:57 s
Completion: 58.11% in 705 m:2 s
Completion: 58.12% in 705 m:8 s
Completion: 58.13% in 705 m:13 s
Completion: 58.14% in 705 m:17 s
Completion: 58.14% in 705 m:24 s
Completion: 58.15% in 705 m:30 s
Completion: 58.16% in 705 m:35 s
Completion: 58.16% in 705 m:42 s
Completion: 58.17% in 705 m:48 s
Completion: 58.18% in 705 m:54 s
Completion: 58.19% in 705 m:59 s
Completion: 58.19% in 706 m:4 s
Completion: 58.2

Completion: 59.82% in 725 m:19 s
Completion: 59.83% in 725 m:24 s
Completion: 59.84% in 725 m:29 s
Completion: 59.84% in 725 m:34 s
Completion: 59.85% in 725 m:39 s
Completion: 59.86% in 725 m:44 s
Completion: 59.87% in 725 m:50 s
Completion: 59.87% in 725 m:56 s
Completion: 59.88% in 726 m:2 s
Completion: 59.89% in 726 m:9 s
Completion: 59.90% in 726 m:16 s
Completion: 59.90% in 726 m:23 s
Completion: 59.91% in 726 m:28 s
Completion: 59.92% in 726 m:34 s
Completion: 59.93% in 726 m:41 s
Completion: 59.93% in 726 m:46 s
Completion: 59.94% in 726 m:53 s
Completion: 59.95% in 726 m:59 s
Completion: 59.95% in 727 m:6 s
Completion: 59.96% in 727 m:10 s
Completion: 59.97% in 727 m:15 s
Completion: 59.98% in 727 m:20 s
Completion: 59.98% in 727 m:24 s
Completion: 59.99% in 727 m:30 s
Completion: 60.00% in 727 m:36 s
Completion: 60.01% in 727 m:41 s
Completion: 60.01% in 727 m:45 s
Completion: 60.02% in 727 m:50 s
Completion: 60.03% in 727 m:55 s
Completion: 60.04% in 728 m:0 s
Completion: 60

Completion: 61.66% in 748 m:55 s
Completion: 61.67% in 749 m:0 s
Completion: 61.68% in 749 m:6 s
Completion: 61.69% in 749 m:15 s
Completion: 61.69% in 749 m:22 s
Completion: 61.70% in 749 m:31 s
Completion: 61.71% in 749 m:37 s
Completion: 61.72% in 749 m:42 s
Completion: 61.72% in 749 m:48 s
Completion: 61.73% in 749 m:54 s
Completion: 61.74% in 749 m:59 s
Completion: 61.74% in 750 m:4 s
Completion: 61.75% in 750 m:12 s
Completion: 61.76% in 750 m:17 s
Completion: 61.77% in 750 m:22 s
Completion: 61.77% in 750 m:27 s
Completion: 61.78% in 750 m:33 s
Completion: 61.79% in 750 m:37 s
Completion: 61.80% in 750 m:42 s
Completion: 61.80% in 750 m:52 s
Completion: 61.81% in 750 m:58 s
Completion: 61.82% in 751 m:2 s
Completion: 61.83% in 751 m:8 s
Completion: 61.83% in 751 m:16 s
Completion: 61.84% in 751 m:23 s
Completion: 61.85% in 751 m:29 s
Completion: 61.86% in 751 m:35 s
Completion: 61.86% in 751 m:40 s
Completion: 61.87% in 751 m:46 s
Completion: 61.88% in 751 m:54 s
Completion: 61.

Completion: 63.51% in 773 m:4 s
Completion: 63.51% in 773 m:9 s
Completion: 63.52% in 773 m:14 s
Completion: 63.53% in 773 m:20 s
Completion: 63.53% in 773 m:26 s
Completion: 63.54% in 773 m:31 s
Completion: 63.55% in 773 m:36 s
Completion: 63.56% in 773 m:41 s
Completion: 63.56% in 773 m:47 s
Completion: 63.57% in 773 m:52 s
Completion: 63.58% in 773 m:57 s
Completion: 63.59% in 774 m:2 s
Completion: 63.59% in 774 m:7 s
Completion: 63.60% in 774 m:11 s
Completion: 63.61% in 774 m:18 s
Completion: 63.62% in 774 m:23 s
Completion: 63.62% in 774 m:28 s
Completion: 63.63% in 774 m:33 s
Completion: 63.64% in 774 m:39 s
Completion: 63.65% in 774 m:45 s
Completion: 63.65% in 774 m:50 s
Completion: 63.66% in 774 m:54 s
Completion: 63.67% in 774 m:59 s
Completion: 63.67% in 775 m:5 s
Completion: 63.68% in 775 m:10 s
Completion: 63.69% in 775 m:16 s
Completion: 63.70% in 775 m:23 s
Completion: 63.70% in 775 m:29 s
Completion: 63.71% in 775 m:32 s
Completion: 63.72% in 775 m:38 s
Completion: 63.